In [1]:
import __init__

Navigated to package root: /home/cyprien/CrystaLLMv2_PKV
Added package root to Python path


### Preprocessing
- Dataset: Materials Project structures (April 2025)
  - Columns: Database, Reduced Formula, CIF, Bandgap (eV), Energy Above Hull (eV)
- Deduplicated, filters applied: 
  - removed negative and null bandgaps
- Cleaned for CIF augmentation, normalisations:
  - Bandgap - power-log ($\beta = 0.8$)
  - $E_{hull}$ - min-max
- Note: I didnt filter to context length here because it was not implemented yet, but filter to context was flagged as True during model training which effectively does the same thing (less efficient)
- Pushed to Huggingface as c-bone/mpdb-2prop_clean

### Training

In [ ]:
models = ['PKV', 'prepend', 'raw', 'slider']
regimes = ['ft', 'scratch']

for models in models:
    for regimes in regimes:
        config_path = f'_config_files/training/conditional/pretraining_benefits/mpdb_{regimes}-{models}.jsonc'

        !torchrun --nproc_per_node=2 _train.py --config config_path

### CIF Generation for each

In [ ]:
for models in models:
    for regimes in regimes:
        gen_config_path = f'_config_files/generation/conditional/pretraining_benefits/mpdb_{regimes}-{models}.jsonc'

        !python _utils/_generating/generate_CIFs.py --config gen_config_path

### Metrics for each

In [ ]:
for models in models:
    for regimes in regimes:
        gen_structs_path = f'_artifacts/pretrain_benefits/{regimes}-methods/mpdb_{regimes}-{models}_gen.parquet'

        postprocessed_path = f'_artifacts/pretrain_benefits/{regimes}-methods/mpdb-{regimes}-{models}_post.parquet'

        !python _utils/_metrics/VUN_metrics.py \
            --input_parquet gen_structs_path \
            --huggingface_dataset 'c-bone/mpdb-2prop_clean' \
            --load_processed_data 'HF-databases/mpdb-2prop_clean/mpdb_2prop_proc.parquet' \
            --output_parquet posprocessed_path \
            --num_workers 32

        !python _utils/_metrics/mace_ehull.py \
            --post_parquet posprocessed_path \
            --output_parquet posprocessed_path \
            --mp_data 'mp_computed_structure_entries.json.gz' \
            --num_workers 16

        !python _utils/_metrics/property_metrics.py \
            --post_parquet posprocessed_path \
            --output_parquet posprocessed_path \
            --property_targets ["Bandgap (eV)", "Energy Above Hull (eV)"] \
            --num_workers 16

### Plotting the results

In [1]:
import __init__
import pandas as pd
from datasets import load_dataset
from __scripts_in_dev._plotting import pretraining_benefits
from _utils import get_metrics_ptnd_vs_scratch

models = ['PKV', 'prepend', 'raw', 'slider']
regimes = ['ft', 'scratch']
dfs = {}
for models in models:
    for regime in regimes:
        if models == 'raw' and regime == 'ft':
            continue
        postprocessed_path = f'_artifacts/pretrain_benefits/{regime}-methods/mpdb-{regime}-{models}_post.parquet'
        # print(pd.read_parquet(postprocessed_path).columns)
        dfs[f'mpdb_{models}_{regime}'] = pd.read_parquet(postprocessed_path)

ds = load_dataset('c-bone/mpdb-2prop_clean', split='train')
train_df = ds.to_pandas()

Navigated to package root: /home/cyprien/CrystaLLMv2_PKV
Added package root to Python path


In [6]:
metrics = get_metrics_ptnd_vs_scratch(
    dfs, 
    train_df=train_df,
    hit_tol_eV=0.5,
    )

hit_rate_vs_density_slope: 0.2761
hit_rate_vs_density_intercept: 0.0261
hit_rate_density_correlation: 0.7804
avg_delta_validity: 0.2042
sem_delta_validity: 0.0213
avg_delta_quality: 0.1435
sem_delta_quality: 0.0085
avg_delta_hit_rate: 0.0263
sem_delta_hit_rate: 0.0050
best_worst_ratio_6_7eV: 9.6875
best_worst_ratio_bandgap: 6.7762
valid_vs_density_slope: 0.0993
valid_vs_density_intercept: 0.4779
valid_density_correlation: 0.1728
quality_vs_density_slope: 0.0504
quality_vs_density_intercept: 0.0722
quality_density_correlation: 0.2164
best_hit_rate_method: PKV
best_hit_rate_pretrained: True
best_hit_rate_value: 0.2258
second_best_hit_rate_method: Prepend
second_best_hit_rate_pretrained: True
second_best_hit_rate_value: 0.2150
hit_rate_improvement_percent: 1.0750
best_valid_method: PKV
best_valid_pretrained: True
best_valid_value: 0.7199
second_best_valid_method: Prepend
second_best_valid_pretrained: True
second_best_valid_value: 0.7087
valid_improvement_percent: 1.1125
best_q_method: Pre